In [2]:
%defaultDatasource jdbc:h2:mem:db

# **MC536 - TRABALHO FINAL**
**Grupo:** Dupla LPT + Dupla JSS
- Gabriel Henrique Rosa Oswaldo - 172185
- Lucas Silva Lopes Do Carmo - 202110
- Gabriel De Alcantara Bomfim Silveira - 197244
- Vitor Coppo Ferreira - 206956

## **Etapa 3: Dupla LPT - Análise relacional com SQL**

### **Fonte de Dados**
Nesta etapa do trabalho utilizamos como fonte de dados o [World Happiness Report 2016](https://www.kaggle.com/unsdsn/world-happiness/version/1#2016.csv), o qual contém pontuações e classificações de felicidade nos países, com um total de 157 instâncias e 13 variáveis. As pontuações são baseadas nas principais perguntas sobre avaliações sobre a vida das pessoas, feitas na pesquisa.

As colunas deste conjunto de dados incluem país, região, rank de felicidade, pontuação de felicidade, intervalo de confiança inferior e superior, Economia (PIB), Família, Saúde (Expectativa de Vida), Liberdade, Confiança no Governo (Corrupção), Generosidade, Distopia residual. O índice de felicidade é determinado por seis fatores - de Economia a Generosidade - que tornam as avaliações de vida mais altas em cada país. 

### **Modelo relacional e lógico**
Com este conjunto de dados, conseguimos obter o seguinte modelo relacional e lógico, respectivamente, apresentados abaixo:

![Modelo Relacional Dupla LPT](./img/modelo_relacional_DuplaLPT.png)

~~~~
Survey (Country, Region, Happiness_Rank,  Happiness_Score, Lower_Confidence_Interval,  
        Upper_Confidence_Interval, Economy, Family,  Life_Expectancy, Freedom, 
        Government_Corruption,  Generosity, Dystopia_Residual)
~~~~

### **Análises em SQL**

### Preparação da tabela

In [11]:
DROP TABLE IF EXISTS Survey;

CREATE TABLE Survey(
    country VARCHAR(50) NOT NULL,
    region VARCHAR(50),
    hap_rank INT NOT NULL,
    hap_score FLOAT,
    lower_confidence_interval FLOAT,
    upper_confidence_interval FLOAT,
    economy FLOAT,
    family FLOAT,
    life_expectancy FLOAT,
    freedom FLOAT,
    gov_corruption FLOAT,
    generosity FLOAT,
    dystopia_residual FLOAT,
    PRIMARY KEY(country)
) AS SELECT 
    Country,
    Region,
    Happiness_Rank,
    Happiness_Score,
    Lower_Confidence_Interval,
    Upper_Confidence_Interval,
    Economy,
    Family,
    Life_Expectancy,
    Freedom,
    Government_Corruption,
    Generosity,
    Dystopia_Residual
FROM CSVREAD('./data/world_happiness_report_2016.csv');

SELECT * FROM Survey;

### Query 1
- Média do happiness score dos países, agrupados por região.

In [12]:
SELECT S.region, AVG(hap_score) avg_hap_score, COUNT(*) countries
    FROM Survey S
    GROUP BY S.region
    ORDER BY avg_hap_score DESC;

### Query 2

- Coeficientes de correlação de Pearson entre a pontuação de felicidade e os seis fatores: Economia, Família, Expectativa de Vida, Liberdade, Confiança no Governo, Generosidade. Essa consulta nos proporciona uma noção de quais dos seis fatores mais influenciam para ter uma pontuação de felicidade mais elevada.

In [13]:
DROP TABLE IF EXISTS CorrelationHapScore;

CREATE TABLE CorrelationHapScore(
    relation VARCHAR(50),
    coeficient FLOAT
);

In [14]:
DROP VIEW IF EXISTS SumHapEcon;
DROP VIEW IF EXISTS HapEcon;

CREATE VIEW HapEcon AS
SELECT S.country, S.hap_score X, S.economy Y, S.hap_score*S.economy XY, S.hap_score*S.hap_score X2, S.economy*S.economy Y2
    FROM Survey S;
    
CREATE VIEW SumHapEcon AS
SELECT SUM(HE.X) SX, SUM(HE.Y) SY, SUM(HE.XY) SXY, SUM(HE.X2) SX2, SUM(HE.Y2) SY2, SUM(HE.X)*SUM(HE.X) QSX, SUM(HE.Y)*SUM(HE.Y) QSY
    FROM HapEcon HE;

INSERT INTO CorrelationHapScore(coeficient, relation)
    VALUES ((SELECT ((157*SHE.SXY) - (SHE.SX*SHE.SY))/SQRT(((157*SHE.SX2) - (SHE.QSX)) * ((157*SHE.SY2) - (SHE.QSY))) 
                FROM SumHapEcon SHE), 'economy');

In [15]:
DROP VIEW IF EXISTS SumHapFam;
DROP VIEW IF EXISTS HapFam;

CREATE VIEW HapFam AS
SELECT S.country, S.hap_score X, S.family Y, S.hap_score*S.family XY, S.hap_score*S.hap_score X2, S.family*S.family Y2
    FROM Survey S;
    
CREATE VIEW SumHapFam AS
SELECT SUM(HE.X) SX, SUM(HE.Y) SY, SUM(HE.XY) SXY, SUM(HE.X2) SX2, SUM(HE.Y2) SY2, SUM(HE.X)*SUM(HE.X) QSX, SUM(HE.Y)*SUM(HE.Y) QSY
    FROM HapFam HE;
    
INSERT INTO CorrelationHapScore(coeficient, relation)
    VALUES ((SELECT ((157*SHE.SXY) - (SHE.SX*SHE.SY))/SQRT(((157*SHE.SX2) - (SHE.QSX)) * ((157*SHE.SY2) - (SHE.QSY))) 
                    FROM SumHapFam SHE), 'family');

In [16]:
DROP VIEW IF EXISTS SumHapLife;
DROP VIEW IF EXISTS HapLife;

CREATE VIEW HapLife AS
SELECT S.country, S.hap_score X, S.life_expectancy Y, S.hap_score*S.life_expectancy XY, S.hap_score*S.hap_score X2, S.life_expectancy*S.life_expectancy Y2
    FROM Survey S;
    
CREATE VIEW SumHapLife AS
SELECT SUM(HE.X) SX, SUM(HE.Y) SY, SUM(HE.XY) SXY, SUM(HE.X2) SX2, SUM(HE.Y2) SY2, SUM(HE.X)*SUM(HE.X) QSX, SUM(HE.Y)*SUM(HE.Y) QSY
    FROM HapLife HE;
    
INSERT INTO CorrelationHapScore(coeficient, relation)
    VALUES ((SELECT ((157*SHE.SXY) - (SHE.SX*SHE.SY))/SQRT(((157*SHE.SX2) - (SHE.QSX)) * ((157*SHE.SY2) - (SHE.QSY))) 
                            FROM SumHapLife SHE), 'life_expectancy');

In [17]:
DROP VIEW IF EXISTS SumHapFree;
DROP VIEW IF EXISTS HapFree;

CREATE VIEW HapFree AS
SELECT S.country, S.hap_score X, S.freedom Y, S.hap_score*S.freedom XY, S.hap_score*S.hap_score X2, S.freedom*S.freedom Y2
    FROM Survey S;
    
CREATE VIEW SumHapFree AS
SELECT SUM(HE.X) SX, SUM(HE.Y) SY, SUM(HE.XY) SXY, SUM(HE.X2) SX2, SUM(HE.Y2) SY2, SUM(HE.X)*SUM(HE.X) QSX, SUM(HE.Y)*SUM(HE.Y) QSY
    FROM HapFree HE;
    
INSERT INTO CorrelationHapScore(coeficient, relation)
    VALUES ((SELECT ((157*SHE.SXY) - (SHE.SX*SHE.SY))/SQRT(((157*SHE.SX2) - (SHE.QSX)) * ((157*SHE.SY2) - (SHE.QSY))) 
                    FROM SumHapFree SHE), 'freedom');

In [18]:
DROP VIEW IF EXISTS SumHapGov;
DROP VIEW IF EXISTS HapGov;

CREATE VIEW HapGov AS
SELECT S.country, S.hap_score X, S.gov_corruption Y, S.hap_score*S.gov_corruption XY, S.hap_score*S.hap_score X2, S.gov_corruption*S.gov_corruption Y2
    FROM Survey S;
    
CREATE VIEW SumHapGov AS
SELECT SUM(HE.X) SX, SUM(HE.Y) SY, SUM(HE.XY) SXY, SUM(HE.X2) SX2, SUM(HE.Y2) SY2, SUM(HE.X)*SUM(HE.X) QSX, SUM(HE.Y)*SUM(HE.Y) QSY
    FROM HapGov HE;
    
INSERT INTO CorrelationHapScore(coeficient, relation)
    VALUES ((SELECT ((157*SHE.SXY) - (SHE.SX*SHE.SY))/SQRT(((157*SHE.SX2) - (SHE.QSX)) * ((157*SHE.SY2) - (SHE.QSY))) 
                            FROM SumHapGov SHE), 'gov_corruption');

In [19]:
DROP VIEW IF EXISTS SumHapGen;
DROP VIEW IF EXISTS HapGen;

CREATE VIEW HapGen AS
SELECT S.country, S.hap_score X, S.generosity Y, S.hap_score*S.generosity XY, S.hap_score*S.hap_score X2, S.generosity*S.generosity Y2
    FROM Survey S;
    
CREATE VIEW SumHapGen AS
SELECT SUM(HE.X) SX, SUM(HE.Y) SY, SUM(HE.XY) SXY, SUM(HE.X2) SX2, SUM(HE.Y2) SY2, SUM(HE.X)*SUM(HE.X) QSX, SUM(HE.Y)*SUM(HE.Y) QSY
    FROM HapGen HE;
    
INSERT INTO CorrelationHapScore(coeficient, relation)
    VALUES ((SELECT ((157*SHE.SXY) - (SHE.SX*SHE.SY))/SQRT(((157*SHE.SX2) - (SHE.QSX)) * ((157*SHE.SY2) - (SHE.QSY))) 
                        FROM SumHapGen SHE), 'generosity');

In [20]:
SELECT * FROM CorrelationHapScore ORDER BY coeficient DESC;

## **Etapa 3: Dupla JSS - Análise relacional com SQL**

### **Fonte de Dados**
https://data.unicef.org/topic/child-health/immunization/  
trends-anual.csv

A base de dados sobre imunização possui estimativas de taxas de imunização de 13 diferentes vacinas em diversos países do mundo nos últimos 10 anos.
Para a análise a seguir, pegamos o recorte do Brasil.
A segunda base foi criada a partir de dados obtidos do [Google Trends](https://trends.google.com/trends/) para pesquisas relacionadas ao movimento anti-vacina na região do Brasil.

### **Modelo relacional e lógico**
Com este conjunto de dados, conseguimos obter o seguinte modelo relacional e lógico, respectivamente, apresentados abaixo:

![Modelo Relacional Dupla JSS](./img/modelo_relacional_DuplaJSS.png)


~~~
Immunization_Estimate(id, unicef_region, iso3, vaccine, Ano, Taxa);
Trends_antivax(Período, Antivax:(Brasil), Vacina Causa autismo(Brasil), Anti-vacina:(Brasil), Anti-vacinação:(Brasil), Soma, Popularidade, Ocorrências);
~~~

## Análise de taxas de vacinação

In [5]:
DROP VIEW IF EXISTS Medias_Vacinas;
DROP VIEW IF EXISTS Medias_Antivax;

DROP TABLE IF EXISTS Antivax_Trends;
DROP TABLE IF EXISTS Immunization_Estimates;

CREATE TABLE Antivax_Trends (
    Periodo INTEGER,
    Popularidade INTEGER,
    PRIMARY KEY(Periodo)
) AS SELECT
    Periodo,
    Popularidade
FROM CSVREAD('./data/trends-antivax-anual.csv');

CREATE TABLE Immunization_Estimates (
    Id INTEGER NOT NULL,
    vaccine VARCHAR,
    Ano INTEGER,
    Taxa INTEGER,
    PRIMARY KEY(Id)
)AS SELECT
    Id,
    vaccine,
    Ano,
    Taxa
FROM CSVREAD('./data/Immunization-estimates-2018-Brasil.csv')

In [6]:
SELECT * FROM Antivax_Trends;
SELECT * FROM Immunization_Estimates;

### **I. Análise de comportamento estatístico**

### 1) Estruturação de view para assitência na análise

In [7]:
CREATE VIEW IF NOT EXISTS Medias_Vacinas AS 
    SELECT vaccine, AVG(Immunization_Estimates.Taxa) Media_taxa
    FROM Immunization_Estimates
    GROUP BY vaccine;
    
SELECT * FROM Medias_Vacinas;

### 2) Para cada ano a quantidade de taxas de imunização acima da média por vacina

In [8]:
DROP VIEW IF EXISTS contador_acima_media;
CREATE VIEW contador_acima_media AS 
    Select Ano Anos_Acima_Da_Media, Immunization_Estimates.vaccine, Immunization_Estimates.taxa, Immunization_Estimates.ano
    From Medias_Vacinas, Immunization_Estimates
    Where Immunization_Estimates.Taxa >= Medias_Vacinas.media_taxa AND Immunization_Estimates.ano = Ano AND Medias_Vacinas.vaccine = Immunization_Estimates.vaccine
    Group by ano, Immunization_Estimates.vaccine;
    
    --para cada ano eu quero selecionar a contagem de valores acima da media

SELECT ano Ano_acima_media, COUNT(*) 
    FROM contador_acima_media
    GROUP BY ano;

SELECT * FROM contador_acima_media;

### 3) Para cada ano a quantidade de taxas de imunização abaixo da média por vacina

In [9]:
DROP VIEW IF EXISTS contador_abaixo_media;
CREATE VIEW contador_abaixo_media AS 
    Select Ano Anos_Abaixo_Da_Media, Immunization_Estimates.vaccine, Immunization_Estimates.taxa, Immunization_Estimates.ano
    From Medias_Vacinas, Immunization_Estimates
    Where Immunization_Estimates.Taxa < Medias_Vacinas.media_taxa AND Immunization_Estimates.ano = Ano AND Medias_Vacinas.vaccine = Immunization_Estimates.vaccine
    Group by ano, Immunization_Estimates.vaccine;
    
    --para cada ano eu quero selecionar a contagem de valores abaixo da media

SELECT ano Ano_abaixo_media, COUNT(*) 
    FROM contador_abaixo_media
    GROUP BY ano;
    
SELECT * FROM contador_abaixo_media;

### 4) Período com maior número de pesquisas contra vacinação

In [10]:
Select Periodo 
    from Antivax_Trends
    WHERE Antivax_Trends.Popularidade = (Select MAX(Popularidade) 
                                             from Antivax_Trends);

2018

## **Etapa 4: Grupo LPT + JSS - Análise hierárquica com XQuery** 

### **Base de Dados e Queries**
Para a realização do trabalho utilizamos os data sets gratuitos disponíveis em: [Orphadata](http://www.orphadata.org/cgi-bin/index.php).   
As bases de dados estão relacionadas a classificação do doenças raras, genes associados a essas doenças, e seus respectivos fenótipos.

Para a elaboração das queries em XQuery nas bases XML utilizamos o programa gratuito [BaseX](http://basex.org/), visto que o programa mostrado pelo professor (http://try.zorba.io/) não suportava o tamanho das fontes de dados.

Abaixo temos todas as queries realizadas, com suas respectivas descrições, bases utilizadas e códigos XQuery:

### **XQuery 01**
**Descrição:** Retorna a quantidade de doenças raras cadastradas na base.  
**Base:** http://www.orphadata.org/data/xml/en_product4_HPO.xml  
**Código:**

~~~XQuery
   for $c in //DisorderList
   
   return count($c/Disorder)
~~~

### **XQuery 02**
**Descrição:** retorna a quantidade de distúrbios associados a doença rara com o id especificado, no caso utilizamos a “Alexander Disease”, que possui o id=2.  
**Base:** http://www.orphadata.org/data/xml/en_product4_HPO.xml  
**Código:**

~~~XQuery
for $c in //Disorder[@id=2]//HPODisorderAssociationList  

return count($c/HPODisorderAssociation)
~~~

### **XQuery 03**
**Descrição:** retorna a lista formatada de distúrbios com a frequência acima de “Frequente”, associadas a doença rara com o id especificado, no caso utilizamos a “Alexander Disease” com frequência "Frequent (79-30%)", "Very frequent (99-80%)" e "Obligate (100%)".<br>
**Base:** http://www.orphadata.org/data/xml/en_product4_HPO.xml<br>
**Código:** 

~~~XQuery
for $c in //Disorder[@id=2]//HPODisorderAssociation

where $c/HPOFrequency[Name="Very frequent (99-80%)"] or

      $c/HPOFrequency[Name="Frequent (79-30%)"]
      
return <sintoma>

         <nome>{data($c//HPOTerm)}</nome>
         
         <frequencia>{data($c/HPOFrequency/Name)}</frequencia>
         
       </sintoma>
~~~

### **XQuery 04**
**Descrição:** retorna uma lista de doenças que possuem o sintoma especificado (HPOTerm="Macrocephaly") com frequência acima de “Frequent (79-30%)”.  
**Base:** http://www.orphadata.org/data/xml/en_product4_HPO.xml  
**Código:**

~~~XQuery
for $c in //Disorder
where $c//HPODisorderAssociation/HPO[HPOTerm="Macrocephaly"] and
      ($c//HPODisorderAssociation//HPOFrequency[Name="Very frequent (99-80%)"] or
       $c//HPODisorderAssociation//HPOFrequency[Name="Frequent (79-30%)"] or
       $c//HPODisorderAssociation//HPOFrequency[Name="Obligate (100%)"])
       
return  data($c/Name)
~~~

### **XQuery 05**
**Descrição:** retorna uma lista de doenças que possuem ambos os sintomas especificados com uma frequência acima de ocasional.  
**Base:** http://www.orphadata.org/data/xml/en_product4_HPO.xml  
**Código:**

~~~XQuery
for $c in (//Disorder),

    $d in (//Disorder)
    
where  $c//HPODisorderAssociation/HPO[HPOTerm="Macrocephaly"] and

       $d//HPODisorderAssociation/HPO[HPOTerm="Nystagmus"] and
       
       ($c[@id] = $d[@id]) 
       
       and
       
      (($c/HPODisorderAssociation/HPOFrequency[Name="Frequent (79-30%)"] or
      
        $c//HPOFrequency[Name="Very frequent (99-80%)"] or
        
        $c//HPOFrequency[Name="Obligate (100%)"]) and
        
        ($d//HPOFrequency[Name="Frequent (79-30%)"] or
        
        $d//HPOFrequency[Name="Very frequent (99-80%)"] or
        
        $d//HPOFrequency[Name="Obligate (100%)"]))
        
        
return data($c/Name)
~~~

## **Etapa 5: Grupo LPT + JSS - Análise de rede com Neo4J/Cypher** 

### **Base de Dados e Queries**

Para a realização da etapa final, utilizamos um data set disponível em: https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016#master.csv  

O data set apresenta taxas de suicídio em diversos países ao longo dos anos entre 1985 e 2016 a partir de suicídios registrados em cada país.

Para a realização das queries, separamos a base em pequenos arquivos menores para podermos correlacioná-los mais facilmente utilizando o programa [Neo4J](https://neo4j.com/).

Abaixo encontram-se as queries realizadas, suas descrições e uma imagem do grafo resultante


### **Query 01**

**Descrição:** importa os países na base  
**Query:**
~~~Cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/gabrieloswaldo/mc536-trabalho/master/jupyter/data/suicidio-paises.csv' AS line
CREATE (:Pais {id: line.Id_pais, name: line.country})
~~~

### **Query 02**

**Descrição:** importa os casos de suicídio  
**Query:**
~~~Cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/gabrieloswaldo/mc536-trabalho/master/jupyter/data/suicidios-casos.csv' AS line CREATE (:Suicidio {id: line.Id, sex: line.sex, age: line.age, generation: line.generation})
~~~

### **Query 03**

**Descrição:** cria a relação entre os casos de suicídio e os países em que aconteceram  
**Query:**  
~~~Cypher
CREATE INDEX ON :Suicidio(id)
CREATE INDEX ON :Pais(name)
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/gabrieloswaldo/mc536-trabalho/master/jupyter/data/suicidios-relations.csv' AS csvLine
MATCH (p:Pais {name: csvLine.country})
MATCH (c:Suicidio {id: csvLine.Id})
CREATE (c)-[:OCORREU {ano: csvLine.year}]->(p)
~~~

![Grafo Query 03](./img/grafo_suicidios_pais.png)

### **Query 04**

**Descrição:** cria um nó para cada geração  
**Query:**
~~~Cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/gabrieloswaldo/mc536-trabalho/master/jupyter/data/suicidios-casos.csv' AS line
MERGE (g:Generation {generation: line.generation})
~~~

### **Query 05**

**Descrição:** cria a relação entre os casos de suicídio e a qual geração a pessoa pertenceu  
**Query:**
~~~Cypher
MATCH (g:Generation)
MATCH (s:Suicidio)
WHERE s.generation = g.generation
CREATE (s)-[:PERTENCE_A]->(g)
~~~

![Grafo Query 05](/img/grafo_suicidios_geracao.png)

### **Query 06**

**Descrição:**  retorna a rede de relações entre os suicídios de pessoas e qual geração ela pertenceu, especificada para dois países.  
**Query:**
~~~Cypher
MATCH (g:Generation {generation:"Boomers"})
MATCH (s:Suicidio)-[:OCORREU]->(p)
WHERE p.name = "Brazil" OR p.name = "Austria"
RETURN (s)-[:PERTENCE_A]->(g), p
~~~

![Grafo Query 06](/img/grafo_comparacao_geracao_2paises.png)

### **Query 07**

**Descrição:** retorna a rede de relações entre os suicídios de pessoas e qual geração ela pertenceu, especificada para três países.  
**Query:**
~~~Cypher
MATCH (g:Generation {generation:"Boomers"})
MATCH (s:Suicidio)-[:OCORREU]->(p)
WHERE p.name = "Cuba" OR p.name = "Macau" OR p.name = "Turkey"
RETURN (s)-[:PERTENCE_A]->(g), p
~~~

![Grafo Query 07](/img/grafo_comparacao_geracao_3paises.png)

### **Query 08**

**Descrição:** cria os nós representantes do sexo masculino (Male) e feminino(Female)  
**Query:**
~~~Cypher
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/gabrieloswaldo/mc536-trabalho/master/jupyter/data/suicidios-casos.csv' AS line
MERGE (s:Sex {sex: line.sex})
~~~

### **Query 09**

**Descrição:** Cria relações entre suicidas Homens com o nó Male e suicidas mulheres com o nó Female.  
**Query:**
~~~Cypher
MATCH (sex:Sex)
MATCH (sui:Suicidio)
WHERE sui.sex = sex.sex
CREATE (sui)-[:TEM_SEXO]->(sex)
~~~

![Grafo Query 09](/img/grafo_sexos.png)

### **Query 10**

**Descrição:** retorna a relação entre suicídios de homens e mulheres em um país.  
**Query:**
~~~Cypher
MATCH (s:Suicidio)-[:OCORREU]->(p:Pais)
MATCH (s:Suicidio)-[:TEM_SEXO]->(sex:Sex)
WHERE p.name = "Macau"
RETURN s, sex, p
~~~

![Grafo Query 10](/img/grafo_diferenca_sexos_paises.png)

### **Query 11**

**Descrição:** roda o pagerank no grafo e retorna o score para os países com mais suicídios  
**Query:**
~~~Cypher
CALL algo.pageRank.stream('Page', 'LINKS', {iterations:20, dampingFactor:0.85})
YIELD nodeId, score
RETURN algo.asNode(nodeId).name AS name,score
ORDER BY score DESC
~~~

![Grafo Query 11](./img/resultado_pagerank.png)